In [1]:
pip install transformers

     |████████████████████████████████| 2.8 MB 13.4 MB/s 
     |████████████████████████████████| 895 kB 36.6 MB/s 
     |████████████████████████████████| 636 kB 35.5 MB/s 
     |████████████████████████████████| 52 kB 999 kB/s 
     |████████████████████████████████| 3.3 MB 37.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [16]:
from transformers import AutoTokenizer, AutoModel
# from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances, linear_kernel
import pandas as pd
import numpy as np
from numpy import savetxt
# from numpy import genfromtxt
import torch
import math

## Define model

In [3]:
#model_name = 'sentence-transformers/bert-base-nli-mean-tokens' # if want to use bert
model_name = 'emilyalsentzer/Bio_ClinicalBERT'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Load categories list 

load pre-defined category list

In [22]:
url = 'https://raw.githubusercontent.com/casszhao/FAIR/main/0901_full_list.csv'
sorted_cat = pd.read_csv(url, header=None)

get a dictionary for embedding looking up later 

In [23]:
sorted_cat = sorted_cat[0].to_list()
sorted_cat = list(dict.fromkeys(sorted_cat))

## Get embeddings (only need to run once)

In [7]:
# process batch, get embeddings for a batch
def get_mean_pooled_np_batch(sorted_cat):
  tokens = {'input_ids': [], 'attention_mask': []}

  for sentence in sorted_cat:
      # encode each sentence and append to dictionary
      new_tokens = tokenizer.encode_plus(sentence, max_length=20,
                                        truncation=True, padding='max_length',
                                        return_tensors='pt')
      tokens['input_ids'].append(new_tokens['input_ids'][0])
      tokens['attention_mask'].append(new_tokens['attention_mask'][0])

  # reformat list of tensors into single tensor, contain all sents
  tokens['input_ids'] = torch.stack(tokens['input_ids'])
  tokens['attention_mask'] = torch.stack(tokens['attention_mask'])
  outputs = model(**tokens)
  embeddings = outputs.last_hidden_state

  attention_mask = tokens['attention_mask']
  mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
  masked_embeddings = embeddings * mask
  summed = torch.sum(masked_embeddings, 1)

  summed_mask = torch.clamp(mask.sum(1), min=1e-9)
  mean_pooled = summed / summed_mask
  return mean_pooled.detach().numpy()

In [10]:
total_batch = math.ceil(len(sorted_cat)/10)

In [12]:
# batch in batch out to save ram
for i in range(total_batch):
  temp_list = sorted_cat[i*10:(i+1)*10]

  if i == 0:
    pooled_emb_array = get_mean_pooled_np_batch(temp_list)
  else:
    temp_pooled_emb_np = get_mean_pooled_np_batch(temp_list)

    pooled_emb_array = np.concatenate((pooled_emb_array, temp_pooled_emb_np), axis=0)

In [17]:
savetxt('candidate_emebdding.csv', pooled_emb_array, delimiter=',')